In [2]:
import pandas as pd
import sys
from goatools import obo_parser
from tqdm import tqdm
import os
import yaml
import h5py

In [89]:
sys.path.append("../")

In [109]:
from scripts.utils import convert_fasta_to_df, filter_df

In [91]:
neg_samples = convert_fasta_to_df("../data/uniprot/notgo_0003723_notgo_0003677_swissprot.fasta")

In [92]:
neg_samples

,identifier,sequence
0,A0A009IHW8,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...
1,A0A023I7E1,MRFQVIVAAATITMITSYIPGVASQSTSDGDDLFVPVSNFDPKSIF...
2,A0A024SC78,MRSLAILTTLLAGHAFAYPKPAPQSVNRRDWPSINEFLSELAKVMP...
3,A0A024SH76,MIVGILTTLATLATLAASVPLEERQACSSVWGQCGGQNWSGPTCCA...
4,A0A026W182,MMKMKQQGLVADLLPNIRVMKTFGHFVFNYYNDNSSKYLHKVYCCV...
...,...,...
45355,X2JAU8,MQPPPRKGNYVKFLKNLHTEQVAKLQLKNQHECDLLEDIRQFTIKR...
45356,X2JDY8,MGGGKNVRRGLEPLEFEECIVDSPDFRENLNRHEKELDHTSHQIKR...
45357,X5JB51,MTERIRARGPRSSSVNSVPLILDIEDFKGDFSFDALFGNLVNDLLP...
45358,X5M5N0,MPDSITNGGRPPAPPSSVSSTTASTTGNFGTRRRLVNRIKKVDELH...


Убрать при предсказании сиквенсы имеющие разметку

In [98]:
train = pd.read_csv("../data/embeddings/input_csv/train_p2.csv")
pdb1000 = pd.read_csv("../data/embeddings/input_csv/pdb1000.csv")
pdb2272 = pd.read_csv("../data/embeddings/input_csv/pdb2272.csv")
pdb20000 = pd.read_csv("../data/embeddings/input_csv/pdb20000.csv")

In [99]:
train_ids = train["identifier"]
pdb1000_ids = pdb1000["identifier"]
pdb2272_ids = pdb2272["identifier"]
pdb20000_ids = pdb20000["identifier"]

In [25]:
path_go = "../data/go_terms/go-basic.obo"
go = obo_parser.GODag(path_go)

../data/go_terms/go-basic.obo: fmt(1.2) rel(2024-06-17) 45,494 Terms


In [26]:
na_term = "GO:0003676"

In [27]:
rec = go[na_term]
na_children_terms = rec.get_all_children()

In [28]:
na_children_terms = list(na_children_terms)

In [29]:
na_children_terms.append(na_term)

In [30]:
data = {'na_terms': na_children_terms}

In [34]:
# # Write the dictionary to a YAML file
# with open(file_path, 'w') as file:
#     yaml.dump(data, file, default_flow_style=False)

In [3]:
def load_dict_from_hdf5(filename):
    """
    Load a dictionary with string keys and NumPy array values from an HDF5 file.

    Parameters:
    filename (str): Name of the HDF5 file to load the data from.

    Returns:
    dict: Dictionary with string keys and NumPy array values.
    """
    loaded_dict = {}
    with h5py.File(filename, "r") as f:
        for key in f.keys():
            loaded_dict[key] = f[key][:]
    return loaded_dict

In [4]:
def save_dict_to_hdf5(data_dict, filename):
    """
    Save a dictionary with string keys and NumPy array values to an HDF5 file.

    Parameters:
    data_dict (dict): Dictionary with string keys and NumPy array values.
    filename (str): Name of the HDF5 file to save the data.
    """
    with h5py.File(filename, "w") as f:
        for key, value in data_dict.items():
            f.create_dataset(key, data=value)

In [169]:
neg_samples = load_dict_from_hdf5("../data/embeddings/ankh_embeddings/neg_samples_2d.h5")

In [170]:
len(neg_samples)

3582

In [6]:
embeddings = load_dict_from_hdf5("../data/embeddings/ankh_embeddings/train_p2_2d.h5")

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '../data/embeddings/ankh_embeddings/train_p2_2d.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [171]:
for key in neg_samples:
    assert key not in embeddings

In [172]:
for key in neg_samples:
    embeddings[key] = neg_samples[key]

In [5]:
save_dict_to_hdf5(embeddings, "../data/embeddings/ankh_embeddings/train_p2_2d.h5")

NameError: name 'embeddings' is not defined

In [150]:
train_p2 = pd.read_csv("../data/embeddings/input_csv/train_p2.csv")
train_p3 = pd.read_csv("../data/embeddings/input_csv/train_p3.csv")

In [155]:
df = train_p3.merge(train_p2, on="identifier", how="left")

In [159]:
neg_samples = df[df["label_y"].isna()]

In [168]:
neg_samples.to_csv("../data/embeddings/input_csv/neg_samples.csv", index=False)

In [165]:
neg_samples.rename(columns={"label_x": "label", "sequence_x": "sequence"}, inplace=True)

/tmp/ipykernel_2692874/3863161955.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_samples.rename(columns={"label_x": "label", "sequence_x": "sequence"}, inplace=True)


In [167]:
neg_samples.drop(columns=["label_y", "sequence_y"], inplace=True)

/tmp/ipykernel_2692874/2760486771.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_samples.drop(columns=["label_y", "sequence_y"], inplace=True)
